---
title: "Skilaverkefni 1"
subtitle: "VÉL407G Hönnun vélbúnaðar - vor 2026"
format:
  hi-template-pdf:
    latex-max-runs: 3
    header-includes:
      - \newcommand{\leftfootertext}{VÉL407G vor 2026}
      - \newcommand{\rightfootertext}{Skilaverkefni 1}
      - \graphicspath{{fig/}}
    include-before-body:
      text: |
        \vspace{-20mm}
        \begin{center}
          Aðferð og rökstudd niðurstaða gildir 70\%.\\
          Frágangur og framsetning lausnar gildir 30\%.
        \end{center}
    titlepage: false
execute:
  echo: false
  output: false
---

# Dæmi 1
Gegnheill öxull með þvermál $d$ verður fyrir bæði jafndreifðri normalspennu $\sigma_n$ vegna áskrafts $P$ sem og beygjuspennu $\sigma_b$ vegna vægis $M$. Hæsta spenna á ytra yfirborði öxuls er $\sigma_{max}=\sigma_n+\sigma_b$.

Áskraftur $P$ hefur meðaltal $\mu_P=779 \text{ kN}$ og staðalfrávik $\hat{\sigma}_P = 72.7\text{ kN}$. Beygjuvægið hefur meðaltal $\mu_M=43.5 \text{ kNm}$ og staðalfrávik $\hat{\sigma}_M=2.53 \text{ kNm}$.

Öxull er úr stáli sem hefur meðalflotstyrk $\bar{S}_y = 553 \text{ MPa}$ og staðalfrávik $\hat{\sigma}_{S_y}=42.7 \text{ MPa}$. 

Gerið ráð fyrir að allar stærðir séu óháðar og normaldreifðar.

Ákvarðið hönnunarstuðul og þvermál öxuls m.v. 99.0\% áreiðanleika (öryggi) gegn flotbjögun.

In [27]:
import math

# converting the given stresses to force and moment
d = 105/1000 # m
A = math.pi*d**2/4
I = math.pi*d**4/64

# mean stresses
mu_a = 90 # MPa
mu_b = 383 # MPa

# stdev
stdev_a = 8.4 # MPa
stdev_b = 22.3 # MPa

# convert axial stress to load P [kN]
mu_p = (mu_a*10**6)*A
stdev_p = (stdev_a*10**6)*A

# convert bending stress to moment M [kNm]
mu_m = (mu_b*10**6)*(I)/(d/2)
stdev_m = (stdev_b*10**6)*(I)/(d/2)

print(f"Cross section area is {A*1000**2:.3f} mm^2")
print(f"Moment of inertia is {I*1000**4:.3f} mm^4")
print(f"Mean force P is {mu_p/1000: .2f} kN")
print(f"Force P stdev is {stdev_p/1000: .3f} kN")
print(f"Mean moment M is {mu_m/1000: .3f} kNm")
print(f"Moment stdev is {stdev_m/1000: .3f} kNm")

Cross section area is 8659.015 mm^2
Moment of inertia is 5966602.352 mm^4
Mean force P is  779.31 kN
Force P stdev is  72.736 kN
Mean moment M is  43.528 kNm
Moment stdev is  2.534 kNm


In [22]:
from scipy.stats import norm
import numpy as np

r = 0.99 # 99% reliability
z = norm.ppf(1-r)
print(z)

# now we need the means and stdevs
mu_p = 779 # kN
stdev_p = 72.7 # kN
mu_m = 43.5 # kNm
stdev_m = 2.53 # kNm

# distribution parameters for sum of stresses
mu_stress = mu_p + mu_m
stdev_stress = np.sqrt(stdev_p**2 + stdev_m**2)

# distribution of strengs
mu_strength = 553 # MPa
stdev_strength = 42.7 # MPa

# help variables C for each random variable
C_stress = stdev_stress/mu_stress
C_strength = stdev_strength/mu_strength

# calculate the required design factor
nd = 1 + np.sqrt(1 - (1 - z**2*C_strength**2)*(1 - z**2*C_stress**2))/(1 - z**2*C_strength**2)
print(nd)

# maximum allowable stress
stress_max = mu_strength/nd
print(stress_max)

# solve the cubic equation - bring all coeffs to base units (Pa, N, Nm)
cubic_coeff = [(stress_max*1e6)*np.pi, 0, -4*(mu_p*1000), -32*(mu_m*1000)]
d_roots = np.roots(cubic_coeff)
print(d_roots)

# only one real solution, which is the d in meters, multiply with 1000 for mm
print(np.real(d_roots[0])*1000)

-2.3263478740408408
1.2796391961192701
432.15306445525374
[ 0.10841016+0.j         -0.05420508+0.08074297j -0.05420508-0.08074297j]
108.41015893449584


# Dæmi 2
Eftirfarandi biti $OABC$ ber álag í $xy$ og $zx$ plönum. Gerið ráð fyrir að gefnar stærðir á mynd hafi a.m.k þrjá markverða tölustafi eða fleiri sé svo sýnt.

(a) Setjið upp Macauley föll fyrir álag, skerkraft og beygjuvægi í $xy$ og $xz$ plönum.
(b) Finnið undirstöðukrafta í $y$ og $z$ stefnur í punktum $O$ og $C$.
(c) Teiknið sker- og vægisferla fyrir bæði $xy$ og $xz$ plön. Berið kennsl á krístíska punkta fyrir skerkraft og beygjuvægi.
(d) Finnið hæstu beygjuspennur (tog og þrýsti) m.v. gefið þversnið og merkið viðeigandi staðsetningar í þvernsiði.

![](v1_d2.png){width=80% fig-align="center"}

\newpage

## Lausn

### (a) Macauley föll
Skoðum $xy$ og $xz$ plönin hvort fyrir sig og setjum upp Macauley föll fyrir viðeigandi stefnur.

Við munum bregða útaf þeim formerkjavenjum sem notaðar eru í Shigley. Formerki á kröftum og vægjum eru miðuð við stefnu þeirra í skilgreindu hnitakerfi. Til þess að það gangi upp, þá eru skerkraftar og vægi skigreind á þversniði bita sem hefur normal í $+x$ stefnu, þ.e. við skerum í bita í punkti $x$ og skoðum krafta og vægi á því þversniði sem er vinstra megin (með normal í $+x$). Formerki á hinu þversniðinu með normal í $-x$ hefur andstæð formerki vegna þriðja lögmáls Newtons.

Látum kraftinn í punkti A vera $\vec{P}_A = [0, 0, P_{Az}]^T$ með $P_{Az}=-1.5 \text{ kN}$ og jafndreifða álagið milli $BC$ vera $\vec{w}_{BC} = [0, w\cos\theta, -w\sin\theta]^T$ með $w=2.00 \text{ kN}$ og $\theta = 30^\circ$.

#### Macauley föll í $xy$ plani
Álagið er $q_y(x)$
$$
    q_y(x) =
        R_{1y} \langle x \rangle^{-1}
        - w \cos\theta \langle x - x_B \rangle^0
        + w \cos\theta \langle x - x_C \rangle^0
        + R_{2y} \langle x - x_C \rangle^{-1}.
$$

Innriskerkraftur $V_y(x)$ á þversnið með normal í $+x$. Við heildum $q_y(x)$ einu sinni með tilliti til $x$ og víxlum formerkjum því jákvæður ytri kraftur veldur neikvæðum innri skerkrafti.
$$
    V_y(x) =
        - R_{1y} \langle x \rangle^0
        + w \cos\theta \langle x - x_B \rangle^1
        - w \cos\theta \langle x - x_C \rangle^1
        - R_{2y} \langle x - x_C \rangle^0.
$$

Að lokum er innra vægið $M_z(x)$ í þversniði í punkti $x$ með normal í $+x$ stefnu fengið með því að heilda $V_y(x)$ einu sinni með tilliti til $x$ og setja formerki miðað við hægri handarreglu og punkt $x$ á bitanum. athugum að jákvætt vægi vegna ytri krafts um punkt $x$ veldur neikvæðu innra vægi $M_z(x)$.
$$
    M_z(x) 
        =
        R_{1y} \langle x \rangle^1
        - \frac{w \cos\theta}{2} \langle x - x_B \rangle^2
        + \frac{w \cos\theta}{2} \langle x - x_C \rangle^2
        + R_{2y} \langle x - x_C \rangle^1
$$

#### Macauley föll í $xz$ plani
Gerum það sama fyrir $xz$ planið.
$$
    q_z(x) 
    = 
        R_{1z} \langle x \rangle^{-1}
        - P_{Az} \langle x - x_A \rangle^{-1}
        - w \sin\theta \langle x - x_B \rangle^0
        + w \sin\theta \langle x - x_C \rangle^0
        + R_{2z} \langle x - x_C \rangle^{-1}.
$$
$$
    V_z(x) 
    =
        - R_{1z} \langle x \rangle^0
        + P_{Az} \langle x - x_A \rangle^0
        + w \sin\theta \langle x - x_B \rangle^1
        - w \sin\theta \langle x - x_C \rangle^1
        - R_{2z} \langle x - x_C \rangle^0.
$$
$$
    M_y(x) 
    =
        - R_{1z} \langle x \rangle^1
        + P_{Az} \langle x - x_A \rangle^1
        + \frac{w \sin\theta}{2} \langle x - x_B \rangle^2
        - \frac{w \sin\theta}{2} \langle x - x_C \rangle^2
        - R_{2z} \langle x - x_C \rangle^1.
$$

### Undirstöðukraftar í punktum $O$ og $C$
Fyrir $xy$ planið þá skoðum við $V_y(x_C)$ til að setja upp kraftajafnvægi í $y$ stefnu $V_y(x_C) = 0$
$$
\begin{aligned}
    V_y(x_C) 
    &=
        - R_{1y}
        + w \cos\theta (x_C - x_B)
        - w \cos\theta (x_C - x_C)
        - R_{2y} \\
    &= 
        - R_{1y}
        + w \cos\theta (x_C - x_B)
        - R_{2y}
    = 0.
\end{aligned}
$$

Fyrir vægisjafnvægi skoðum við $M_z(x_C) = 0$ sem er jafngilt því að skoða vægisjafnvægi um punkt $C$.
$$
\begin{aligned}
    M_z(x_c) 
    &= 
        R_{1y}x_C
        - \frac{w \cos\theta}{2} (x_C - x_B)^2
        + \frac{w \cos\theta}{2} (x_C - x_C)^2
        + R_{2y} ( x_C - x_C ) \\
    &= 
        R_{1y}x_C
        - \frac{w \cos\theta}{2} (x_C - x_B)^2
    = 0.
\end{aligned}
$$

Vægisjafnvægi gefur að
$$
    R_{1y} = \frac{w \cos\theta}{2x_C} ( x_C - x_B )^2.
$$

Þá fáum við úr kraftajafnvægi að 
$$
\begin{aligned}
    R_{2y} 
    &= 
        - R_{1y}
        + w \cos\theta (x_C - x_B) \\
    &=
        -\frac{w \cos\theta}{2x_C} ( x_C - x_B )^2
        + w \cos\theta (x_C - x_B).
\end{aligned}
$$

Úr $xz$ plani fáum við svipað með því að setja $V_z(x_C)=0$ og $M_y(x_C)=0$
$$
\begin{aligned}
    V_z(x_C) 
    &=
        - R_{1z}
        + P_{Az}
        + w \sin\theta (x_C - x_B)
        - w \sin\theta (x_C - x_C)
        - R_{2z}\\
    &=
        - R_{1z}
        + P_{Az}
        + w \sin\theta (x_C - x_B)
        - R_{2z}
    = 0.
\end{aligned}
$$

$$
\begin{aligned}
    M_y(x) 
    &=
        - R_{1z} x_C
        + P_{Az} (x_C - x_A)
        + \frac{w \sin\theta}{2} (x_C - x_B)^2
        - \frac{w \sin\theta}{2} (x_C - x_C)^2
        - R_{2z} (x_C - x_C) \\
    &=
        - R_{1z} x_C
        + P_{Az} (x_C - x_A)
        + \frac{w \sin\theta}{2} (x_C - x_B)^2
    = 0.
\end{aligned}
$$

Vægisjafnvægi gefur 
$$
    R_{1z}
    =  
        \frac{1}{x_C}
        \left( 
            P_{Az} (x_C - x_A)
        + \frac{w \sin\theta}{2} (x_C - x_B)^2
        \right).
$$

Kraftajafnvægi gefur
$$
\begin{aligned}
    R_{2z} 
    &=
        -R_{1z}
        + P_{Az}
        + w \sin\theta (x_C - x_B)
\end{aligned}
$$

In [45]:
#| echo: true
#| output: true

# reiknum undirstöðukrafta
import numpy as np

# gefnar stærðir
theta = 30*np.pi/180 # rad
P_Az = 1500 # N
w = 2000 # N

# hnit punkta
x_0 = 0
x_A = 0.5
x_B = x_A + 1
x_C = x_B + 1.5

# undirstöðukraftar í y stefnu
R_1y = (
    (w * np.cos(theta)) 
    * (x_C - x_B)**2 
    / (2 * x_C)
)

R_2y = (
    -R_1y
    + w * np.cos(theta) * (x_C - x_B)
)
# undirstöðukraftar í z stefnu
R_1z = (
    (
        P_Az * (x_C - x_A) 
        + w * np.sin(theta) 
        * (x_C - x_B)**2 / 2
    )
    / x_C
)

R_2z = (
    - R_1z
    + P_Az
    + w * np.sin(theta) * (x_C - x_B)
)

# söfnum saman
R1 = (R_1y, R_1z)
R2 = (R_2y, R_2z)

# prentum
print(f"Undirstöðukraftar í punkti 1, R1=(R_1y, R_1z)=({R_1y:.2f}, {R_1z:.2f}) N")
print(f"Undirstöðukraftar í punkti 2, R2=(R_2y, R_2z)=({R_2y:.2f}, {R_2z:.2f}) N")

# athugum niðurstöðu m.v. jöfnur fyrir krafta og vægi
# skilgreinum fyrst einvítt macauley fall
mac = lambda x, a, n: 0 if (n < 0) or (x < a) else (x - a) ** n

# skilgreinum V(x) og M(x) sem vigurföll
def V_fun(x, R1, R2): 
    Vy = (
        - R1[0] * mac(x, x_0, 0)
        + w * np.cos(theta) * mac(x, x_B, 1)
        - w * np.cos(theta) * mac(x, x_C, 1)
        - R2[0] * mac(x, x_C, 0)
    )

    Vz = (
        - R1[1] * mac(x, x_0, 0)
        + P_Az * mac(x, x_A, 0)
        + w * np.sin(theta) * mac(x, x_B, 1)
        - w * np.sin(theta) * mac(x, x_C, 1)
        - R2[1] * mac(x, x_C, 0)
    )

    V = np.array(
        [Vy, Vz]
    )
    return V

def M_fun(x, R1, R2): 
    My = (
        - R1[1] * mac(x, x0, 1)
        + P_Az * mac(x, xA, 1)
        + w * np.sin(theta) * mac(x, x_B, 2) / 2
        - w * np.sin(theta) * mac(x, x_C, 2) / 2
        - R2[1] * mac(x, xC, 1)
    )

    Mz = (
        + R1[0] * mac(x, x0, 1)
        - w * np.cos(theta) * mac(x, x_B, 2) / 2
        + w * np.cos(theta) * mac(x, x_C, 2) / 2
        + R2[0] * mac(x, xC, 1)
    )

    M = np.array(
        [My, Mz]
    )
    return M

# reiknum V(xC) og M(xC) tékkum að allt sé =0
print(f"Kraftajafnvægi V(x_C)= {V_fun(x_C, R1, R2)} N")
print(f"Vægisjafnvægi M(x_C)= {M_fun(x_C, R1, R2)} Nm")


Undirstöðukraftar í punkti 1, R1=(R_1y, R_1z)=(649.52, 1625.00) N
Undirstöðukraftar í punkti 2, R2=(R_2y, R_2z)=(1948.56, 1375.00) N
Kraftajafnvægi V(x_C)= [0. 0.] N
Vægisjafnvægi M(x_C)= [-2.27373675e-13 -2.27373675e-13] Nm


Svo útreiknaðir undirstöðukraftar eru
$$
    \vec{R}_1 = 
    \begin{bmatrix}
        R_{1y} \\ 
        R_{1z}
    \end{bmatrix}
    =
    \begin{bmatrix}
        0.650 \\ 
        1.63 
    \end{bmatrix} \text{ kN},
$$
og
$$
    \vec{R}_2 = 
    \begin{bmatrix}
        R_{2y} \\ 
        R_{2z}
    \end{bmatrix}
    =
    \begin{bmatrix}
        1.95 \\ 
        1.38 
    \end{bmatrix} \text{ kN}.
$$  

### (a) Macauley föll - önnur aðferð
Við getum nýtt okkur kosti þess að formerki krafta og vægja fylgja því hnitakerfi sem er skilgreint og skrifað upp Macauley föllin með þrívíðum vigrum í $xyz$ plani. Með því getum við soðið niður þessi sex Macauley föll úr fyrri aðferð niður um helming í þrjú föll.

Verkefnið er aðeins einfaldað af þeim sökum að bitinn er einvíður (liggur á $x$ ás) og allir kraftar liggja í $yz$ planinu, þ.e. engir áskraftar í $\pm x$ stefnur.

Látum undirstöðurkrafta vera vigra $\vec{R}_1$ og $\vec{R}_2$. Látum einnig álagið í punkti $A$ vera $\vec{P}_A = [0, 0, P_{Az}]^T$ með $P_{Az}=-1.5 \text{ kN}$ og jafndreifða álagið milli $BC$ vera $\vec{w}_{BC} = [0, w\cos\theta, w\sin\theta]^T$ með $w=-2.00 \text{ kN/m}$ og $\theta = 30^{\circ}$. Athugið að hér látum við stefnu vigrana vera stjórnað af formerki skalargildanna sem mynda vigranna og setjum ekki formerki inn í vigrana sjálfa.

Álagið $\vec{q}$ má þá rita sem
$$
    \vec{q}(x) 
    =
    \vec{R}_1 \langle x \rangle^{-1}
    + \vec{P}_A \langle x - x_A \rangle^{-1}
    + \vec{w}_{BC} \langle x - x_B \rangle^0
    - \vec{w}_{BC} \langle x - x_C \rangle^0
    + \vec{R}_2 \langle x - x_C \rangle^{-1}.
$$

Innri skerkrafturinn $\vec{V}(x)$ fæst með því að heilda Macauley föllin í $\vec{q}(x)$ einu sinni og víxla formerkjum vigranna
$$
    \vec{V}(x)
    =
    - \vec{R}_1 \langle x \rangle^0
    - \vec{P}_A \langle x - x_A \rangle^0
    - \vec{w}_{BC} \langle x - x_B \rangle^1
    + \vec{w}_{BC} \langle x - x_C \rangle^1
    - \vec{R}_2 \langle x - x_C \rangle^0.
$$

Innra vægið $\vec{M}$ þarf að nálgað aðeins öðruvísi fyrir en áður því vægið er krossmargfeldi stefnuvigurs frá punkti $x$ í átakspunkt krafts og ytri kraftavigra. Við getum því ekki heildað eingöngu Macauley föllin í $\vec{V}(x)$ einu sinni til að fá út $\vec{M}(x)$, því $\vec{V}(x)$ lýsir innri kröftum en ekki ytri kröftum.

Hins vegar inniheldur $\vec{V}(x)$ allar upplýsingar um krafta sem við þurfum ef formerkjum er víxlað. Að auki eru engin stök vægi sem þarf að bæta við óbreyttum.

Fyrir ákveðinn kraft $F$ sem verkar í punkti $p$ þá er stefnuvigur í þann punkt úr punkti $x$ með $x \geq p$ vigurinn $\vec{r} = (p - x)\hat{i} = -(x - p)\hat{i} = -\langle x - p \rangle^1 \hat{i}$ þar sem $\hat{i}$ er einingavigurinn samsíða $x$ ás.

Þar með getum við smíðað fallið $\vec{M}(x)$ fyrir innra vægið í punkti $x$ sem
$$
\begin{aligned}
    \vec{M}(x)
    &=
    \left ( -\langle x \rangle^1 \hat{i} \times \vec{R}_1 \langle x \rangle^0 \right) \\
    &+ \left( -\langle x - x_A \rangle^1 \hat{i} \times \vec{P}_A \langle x - x_A \rangle^0 \right) \\
    &+ \left( -\frac{1}{2} \langle x - x_B \rangle^1 \hat{i} \times \vec{w}_{BC} \langle x - x_B \rangle^1 \right) \\
    &- \left( -\frac{1}{2} \langle x - x_C \rangle^1 \hat{i} \times \vec{w}_{BC} \langle x - x_C \rangle^1 \right) \\
    &+ \left( -\langle x - x_C \rangle^1 \hat{i} \times \vec{R}_2 \langle x - x_C \rangle^0 \right). \\
\end{aligned}
$$

Hægt væri að losna við krossmargfeldin með því að skoða vigrana og beita hægri-handar reglu - sem var einmitt það sem var notað í fyrri útgáfu lausnar. Þessi krossmargfeldi má frekar forrita og reikna tölulega. Kosturinn hér er sá að formerki á vægi um $y$ og $z$ ás koma út úr krossmargfeldunum frekar en frá okkur sjálfum við beitingu hægri-handar reglu.

Við leysum fyrir undirstöðurkrafta sem fyrr með því að setja $\vec{V}(x_C) = 0$ og $\vec{M}(x_C)=0$, sem gefur
$$
    - \vec{R}_1
    - \vec{P}_A
    - \vec{w}_{BC} (x_C - x_B)
    - \vec{R}_2
    =
    0
$$
og
$$
\begin{aligned}
    -x_C \hat{i} \times \vec{R}_1 x_C 
    =
    &- \left( -(x_C - x_A) \hat{i} \times \vec{P}_{A} \right) \\
    &- \left( -\frac{1}{2} (x_C - x_B)\hat{i} \times \vec{w}_{BC} (x_C - x_B) \right)
\end{aligned}
$$
Þar sem $\vec{R}_1 = [0, R_{1y}, R_{1z}]^T$ og $\hat{i}=[1, 0, 0]^T$ þá er $-\hat{i} \times \vec{R}_1 = [0, R_{1z}, -R_{1y}]^T$. Við getum skalað báða vigra með $x_C$ og fengið $-x_C\hat{i} \times x_C\vec{R}_1= x_C[0, R_{1z}, -R_{1y}]^T$. Þar með fæst lausnin
$$
\begin{aligned}
    \begin{bmatrix}
        0 \\
        R_{1z} \\
        -R_{1y}
    \end{bmatrix}
    &=
    \frac{1}{x_C}
    \left[
        - \left( -(x_C - x_A) \hat{i} \times \vec{P}_{A} \right)
        - \left( -\frac{1}{2} (x_C - x_B)\hat{i} \times \vec{w}_{BC} (x_C - x_B) \right)
    \right] \\
    &= 
    \frac{1}{x_C}
    \left[
        - \left( (x_A - x_C) \hat{i} \times \vec{P}_{A} \right)
        - \left( \frac{1}{2} (x_B - x_C)\hat{i} \times \vec{w}_{BC} (x_C - x_B) \right)
    \right]
\end{aligned}  
$$

Við getum lagað til hnitin í lausninni til að stemma við $\vec{R}_1 = [0, R_{1y}, R_{1z}]$ og leyst svo fyrir $\vec{R}_2$
$$
    \vec{R}_2
    = 
    - \vec{R}_1
    - \vec{P}_A
    - \vec{w}_{BC} (x_C - x_B)
$$

In [52]:
#| echo: true
#| output: true

import numpy as np

# skilgreinum einvítt Macauley fall
mac = lambda x, a, n: 0 if (n < 0) or (x < a) else (x - a) ** n

# skilgreinum gefnar stærðir
P_A = np.array([0, 0, -1500]) # N
w = 2000 # N/m
theta = 30 * np.pi / 180 # radians
w_BC = np.array([0, -w*np.cos(theta), -w*np.sin(theta)]) # N/m

# einingavigur í x
vec_i = np.array([1, 0, 0])

# skilgreinum V(x) og M(x) sem föll
def V_fun(x, R_1, R_2): 
    V = (
        - R_1 * mac(x, x_0, 0)
        - P_A * mac(x, x_A, 0)
        - w_BC * mac(x, x_B, 1)
        + w_BC * mac(x, x_C, 1)
        - R_2 * mac(x, x_C, 0)
    )
    return V

def M_fun(x, R_1, R_2):
    M = (
        np.cross(-mac(x, x_0, 1) * vec_i, R_1 * mac(x, x_0, 0))
        + np.cross(-mac(x, x_A, 1) * vec_i, P_A * mac(x, x_A, 0))
        + np.cross(-mac(x, x_B, 1) * vec_i, w_BC * mac(x, x_B, 1)) / 2
        - np.cross(-mac(x, x_C, 1) * vec_i, w_BC * mac(x, x_C, 1)) / 2
        + np.cross(-mac(x, x_C, 1) * vec_i, R_2 * mac(x, x_C, 0,))

    )
    return M

# reiknum undirstöðukrafta skv V(x) og M(x)
R_1 = (
    (
        - np.cross(-(x_C - x_A) * vec_i, P_A)
        - np.cross(-0.5 * (x_C - x_B) * vec_i, w_BC * (x_C - x_B))
    )
    / x_C
)

# víxlum til í R_1
R_1 = np.array([0, -R_1[2], R_1[1]])

# leysum fyrir R_1
R_2 = -R_1 - P_A - w_BC * (x_C - x_B)

print(f"Prentum niðurstöður og staðfestum jafnvægi V(x_C)=0 og M(x_C)=0")
print(f"R_1 = {R_1} N")
print(f"R_2 = {R_2} N")
print(f"V_fun(x_C, R_1, R_2) = {V_fun(x_C, R_1, R_2)} N")
print(f"M_fun(x_C, R_1, R_2) = {M_fun(x_C, R_1, R_2)} Nm")


Prentum niðurstöður og staðfestum jafnvægi V(x_C)=0 og M(x_C)=0
R_1 = [   0.          649.51905284 1625.        ] N
R_2 = [  -0.         1948.55715851 1375.        ] N
V_fun(x_C, R_1, R_2) = [0. 0. 0.] N
M_fun(x_C, R_1, R_2) = [0.00000000e+00 2.27373675e-13 0.00000000e+00] Nm


### (c) Sker- og vægisferlar
Teiknum ferla með python m.v. skilgreind föll fyrir $V(x)$ og $M(x)$ úr lið (b).

In [ ]:
#| echo: true
#| output: true

# teiknum sker og vægisferla
import matplotlib as plt

# teiknum V(x) og M(x) yfir bilið [0, xC]
x_vals = np.linspace(0, x_C, 1000)
V_vals = np.array([V_fun(x, R1, R2) for x in x_vals])
M_vals = np.array([M_fun(x, R1, R2) for x in x_vals])

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(x_vals, V_vals[:, 0], label='V_y(x)')
plt.plot(x_vals, V_vals[:, 1], label='V_z(x)')
plt.title('Skerkraftsferlar V(x) í y og z stefnur')
plt.xlabel('x (m)')
plt.ylabel('Skerkraftur (N)')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.legend()
plt.grid()
plt.subplot(2, 1, 2)
plt.plot(x_vals, M_vals[:, 1], label='M_z(x)')
plt.plot(x_vals, M_vals[:, 0], label='M_y(x)')
plt.title('Beygjuvægi M(x) um y og z ás')
plt.xlabel('x (m)')
plt.ylabel('Beygjuvægi (Nm)')
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


TypeError: bad operand type for unary -: 'tuple'

# Dæmi 3
Öxull $ABD$ hefur gegnheilt hringlaga þversnið með þvermál $d=12.0 \text{ mm}$. Stangir $BE$ og $CD$ eru soðnar á öxul. Gerið ráð fyrir að gefnar stærðir á mynd hafi a.m.k þrjá markverða tölustafi eða fleiri sé svo sýnt.

(a) Ákvarðið undirstöðukrafta og vægi í punkti A.
(b) Teiknið skerkrafts og vægisferla.
(c) Ákvarðið staðsetningu á krítískum spennum í þversniði öxuls í punkti A.
(d) Ákvarðið höfuðspennur ásamt samsvarandi hornum höfuðplana og hæstu skerspennur í krítískum punkti úr (c) lið.

![](v1_d3.png){width=80% fig-align="center"}

\newpage

# Dæmi 4
Biti $OAB$ er innspenntur í punkti $O$ og myndaður með því að festa saman tvo rétthyrnda vinkla sem báðir hafa jafnar hliðarlengdir $w=125 \text{ mm}$ og þykkt $t=10.0 \text{ mm}$. Á bitann verkar jafndreift álag $q=1.00 \text{ kN/m}$ yfir $OAB$ og stakur kraftur $P_A=2.50 \text{ kN}$ í punkti A. Gerið ráð fyrir að allar gefnar stærðir á mynd hafi a.m.k þrjá markverða tölustafi eða fleiri sé svo sýnt.

(a) Ákvarðið lóðrétta færslu $y(x)$ í punkti $x=B$ með samlagningaraðferð (e. superposition).
(b) Endurtakið lið (a) en notið Macauley sérstöðuföll.
(c) Berið saman niðurstöður í (a) og (b). 
(d) Teiknið $y(x)$ og $\theta(x)$ þar sem $\theta$ er halli (e. slope) bitans.

![](v1_d4.png){width=80% fig-align="center"}